<a href="https://colab.research.google.com/github/ReiAkio/AI_Project_Semester2/blob/main/Maua_AI_Projeto_Semestre2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Filtro dos dados

### Import das Bibliotecas

In [13]:
import librosa
import numpy as np
import os
import re
import wave


from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

import requests
import json

### Carregar arquivos de audio do path especificado e aplicar STFT

In [3]:
def process_audio_file(file_path):
    # Carregar o arquivo de áudio
    y, sr = librosa.load(file_path, mono=True)
    # Calcular o STFT
    stft = np.abs(librosa.stft(y))
    return stft

### Extraindo arquivos de audio com STFT

In [4]:
%pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=cb075266c5125e3318be0a2ff4a4bb73fcd3f05634661fe1dd17edf6cda057b1
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [5]:
# Função para extrair características do STFT (frequência dominante e outras)
def extract_features(stft, y, sr):
    # Calcular frequências para cada bin da FFT
    freqs = librosa.fft_frequencies(sr=22050, n_fft=2048)
    # Calcular a média do STFT em cada frequência
    mean_stft = np.mean(stft, axis=1)
    # Encontrar a frequência dominante
    dominant_freq = freqs[np.argmax(mean_stft)]

    # Calcular MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Calcular o espectrograma de Mel
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

    # Calcular o cromagrama
    chromagram = librosa.feature.chroma_stft(S=stft, sr=sr)

    # Calcular o contraste espectral
    spectral_contrast = librosa.feature.spectral_contrast(S=stft, sr=sr)

    # Calcular o Rolloff espectral
    spectral_rolloff = librosa.feature.spectral_rolloff(S=stft, sr=sr)

    # Calcular o Tom médio
    pitch = librosa.pitch_tuning(y)
    mean_pitch = np.mean(pitch)

    # Calcular o Ponto culminante do Spectrograma
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

    # Filtrar o sinal harmônico e percussivo
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    # Calcular o STFT para o sinal harmônico
    stft_harmonic = librosa.stft(y_harmonic)

    # Calcular o STFT para o sinal percussivo
    stft_percussive = librosa.stft(y_percussive)

    # Concatenar todas as características extraídas
    features = np.concatenate((np.mean(mfccs, axis=1),
                               np.mean(mel_spectrogram, axis=1),
                               np.mean(chromagram, axis=1),
                               np.mean(spectral_contrast, axis=1),
                               [dominant_freq],
                               [np.mean(spectral_rolloff)],
                               [mean_pitch],
                               [np.max(spec_centroid)]))

    return features

## Aprendizado Supervisionado

### Rotulando por caminho

In [6]:
def get_label_from_path(file_path):
    # Extracts the label 'speaker' from 'speakerX.wav'
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    # Use regular expression to remove trailing digits
    label = re.sub(r'\d+', '', base_name)
    return label

### Organizando os dados para treinos e testes

In [7]:
def prepare_data(audio_files):
    X = []
    y = []
    for file_path, label in audio_files:

        y_data, sr = librosa.load(file_path)
        stft = process_audio_file(file_path)

        features = extract_features(stft,y_data, sr)


        X.append(features)
        y.append(label)

    X = np.array(X)
    y = np.array(y)

    print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")
    return X, y

In [11]:
def load_data_from_directory(directory_path):
    audio_files = []

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.wav') or file.endswith('.mp3'):
                file_path = os.path.join(root, file)
                label = get_label_from_path(file_path)
                audio_files.append((file_path, label))

    print(f"Number of audio files: {len(audio_files)}")
    return prepare_data(audio_files)


### Normalização

In [10]:
def normalization(X_train,X_test):
    global scaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

### Aplicando Logistic Regression

In [14]:
def logistic_regression_model(X_train,X_test,y_train,y_test):
    global logreg_model

    # Logistic Regression
    logreg_model = LogisticRegression()
    logreg_model.fit(X_train, y_train)
    logreg_predictions = logreg_model.predict(X_test)
    print('Logistic Regression Accuracy: ', accuracy_score(y_test, logreg_predictions))
    print('Classification Report for logistic Regression Model:')
    print(classification_report(y_test, logreg_predictions))

### Aplicando SVM

In [15]:
def svm_model(X_train, X_test, y_train, y_test):
    svm_clf = SVC()
    svm_clf.fit(X_train, y_train)
    svm_predictions = svm_clf.predict(X_test)
    print('SVM Accuracy: ', accuracy_score(y_test, svm_predictions))
    print('Classification Report for SVM Model:')
    print(classification_report(y_test, svm_predictions))

### Aplicando KNN

In [16]:
def knn_model(X_train, X_test, y_train, y_test):
    knn_clf = KNeighborsClassifier()
    knn_clf.fit(X_train, y_train)
    knn_predictions = knn_clf.predict(X_test)
    print('KNN Accuracy: ', accuracy_score(y_test, knn_predictions))
    print('Classification Report for KNN Model:')
    print(classification_report(y_test, knn_predictions))

### Aplicando Neural Network

In [17]:
def neural_network_model(X_train, X_test, y_train, y_test):
    nn_clf = MLPClassifier()
    nn_clf.fit(X_train, y_train)
    nn_predictions = nn_clf.predict(X_test)
    print('Neural Network Accuracy: ', accuracy_score(y_test, nn_predictions))
    print('Classification Report for Neural Network Model:')
    print(classification_report(y_test, nn_predictions))